<a href="https://colab.research.google.com/github/dgizdevans/master/blob/main/ai_project/object_detection_yolov8_training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Remove previous unstable version of ultralytics
!pip uninstall -y ultralytics

In [ ]:
# Install libraries
!pip install ultralytics==8.0.3
!pip install google-cloud-storage

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.0/117.0 kB 14.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 247.5/247.5 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.5/154.5 kB 17.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 79.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.5/79.5 kB 9.3 MB/s eta 0:00:00
  Created wheel for antlr4-python3-runtime: filename=antlr4_python3_runtime-4.9.3-py3-none-any.whl size=144555 sha256=15475d14b5b49d61641f2e319367826625e39b9ffa4e30a5f1e48bfa1068bcbd
  Stored in directory: /root/.cache/pip/wheels/12/93/dd/1f6a127edc45659556564c5730f6d4e300888f4bca2d4c5a88
Successfully built antlr4-python3-runtime


In [ ]:
# Import necessary modules
import os
from google.colab import auth
from google.cloud import storage
from ultralytics import YOLO

In [ ]:
# Authenticate and connect to GCP
auth.authenticate_user()
project_id = "ai-group-project"  # Your Google Cloud Project ID
client = storage.Client(project=project_id)
bucket_name = "ai-group-project-data"  # Your GCP bucket name
bucket = client.bucket(bucket_name)

In [ ]:
# Local paths for storing data
local_dataset_path = "/content/datasets/labeled_data"
local_data_yaml = f"{local_dataset_path}/data.yaml"
gcs_data_yaml_path = "datasets/model/data.yaml"  # Path to data.yaml in GCP
gcs_labeled_data_path = "datasets/model"  # Path to labeled data (images and labels)

In [ ]:
# Create local directories
os.makedirs(local_dataset_path, exist_ok=True)

In [ ]:
# Function to download data from GCP
def download_from_gcs(bucket, gcs_folder, local_folder):
    blobs = bucket.list_blobs(prefix=gcs_folder)
    for blob in blobs:
        # Skip folders
        if blob.name.endswith('/'):
            continue
        local_file_path = os.path.join(local_folder, os.path.relpath(blob.name, gcs_folder))
        os.makedirs(os.path.dirname(local_file_path), exist_ok=True)
        blob.download_to_filename(local_file_path)

In [ ]:
# Download labeled data from GCP
print("Downloading labeled data from GCP...")
download_from_gcs(bucket, gcs_labeled_data_path, local_dataset_path)
print("Labeled data downloaded successfully!")

Labeled data downloaded successfully!


In [ ]:
## Check data.yaml status and adapt paths to the local environment
import yaml

# Check if data.yaml exists
if not os.path.exists(local_data_yaml):
    raise FileNotFoundError(f"{local_data_yaml} not found. Ensure your data.yaml is uploaded to GCP.")

# Display the contents of data.yaml
print("Found data.yaml. Displaying its contents:")
with open(local_data_yaml, 'r') as yaml_file:
    data_yaml_content = yaml.safe_load(yaml_file)
    print(yaml.dump(data_yaml_content, default_flow_style=False))

Found data.yaml. Displaying its contents:
names:
  0: V
  1: C
  2: S
train: /content/datasets/labeled_data/train
val: /content/datasets/labeled_data/val



In [ ]:
# Define paths for pretrained model and training results
pretrained_model_path = "yolov8n.pt"  # Pretrained YOLOv8 model
output_dir = "/content/runs/train"  # Directory for saving training results

In [ ]:
# Train the YOLOv8 model
print("Starting model training...")
model = YOLO(pretrained_model_path)
model.train(
    data=local_data_yaml,  # Path to data.yaml
    epochs=50,            # Number of epochs
    imgsz=640,            # Image size
    batch=16,             # Batch size
    save=True,            # Save model
    project=output_dir,   # Directory for saving results
    name="yolov8_training"  # Experiment name
)
print("Model training completed.")

Starting model training...


100%|██████████| 6.23M/6.23M [00:00<00:00, 132MB/s]

/usr/local/lib/python3.10/dist-packages/ultralytics/nn/tasks.py:341: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  ckpt 

Model training completed.


In [ ]:
# Explicit validation after training
print("Starting model validation...")
val_results = model.val(data=local_data_yaml)
print("Validation completed.")

Ultralytics YOLOv8.0.3 🚀 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (NVIDIA A100-SXM4-40GB, 40514MiB)
Fusing layers... 


Starting model validation...


Model summary: 168 layers, 3006233 parameters, 13065 gradients, 8.1 GFLOPs
val: Scanning /content/datasets/labeled_data/val/labels.cache... 163 images, 0 backgrounds, 0 corrupt: 100%|██████████| 163/163 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95):   9%|▉         | 1/11 [00:00<00:07,  1.29it/s]Exception in thread Thread-182 (plot_images):
Traceback (most recent call last):
  File "/usr/lib/python3.10/threading.py", line 1016, in _bootstrap_inner
    self.run()
  File "/usr/lib/python3.10/threading.py", line 953, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/local/lib/python3.10/dist-packages/ultralytics/yolo/utils/plotting.py", line 250, in plot_images
    annotator.box_label(box, label, color=color)
  File "/usr/local/lib/python3.10/dist-packages/ultralytics/yolo/utils/plotting.py", line 63, in box_label
    w, h = self.font.getsize(label)  # text width, height
AttributeError: 'FreeTypeFont' object has no

Validation completed.


In [ ]:
# Save training results back to GCP
gcs_output_path = "models/yolov8_training"  # Path for saving results in GCP
print("Uploading training results to GCP...")
output_training_path = os.path.join(output_dir, "yolov8_training")
blobs_to_upload = [os.path.join(dp, f) for dp, dn, filenames in os.walk(output_training_path) for f in filenames]

for local_file in blobs_to_upload:
    relative_path = os.path.relpath(local_file, output_training_path)
    gcs_file_path = os.path.join(gcs_output_path, relative_path)
    blob = bucket.blob(gcs_file_path)
    blob.upload_from_filename(local_file)
print(f"Training results uploaded to GCP at {gcs_output_path}")

Uploading training results to GCP...
Training results uploaded to GCP at models/yolov8_training


In [ ]:
# Notifications for completion of steps
print("\nProcess Summary:")
print("1. Labeled data downloaded successfully.")
print("2. YOLOv8 model training completed.")
print("3. Model validation completed with results displayed.")
print("4. Training results successfully uploaded to GCP.")


Process Summary:
1. Labeled data downloaded successfully.
2. YOLOv8 model training completed.
3. Model validation completed with results displayed.
4. Training results successfully uploaded to GCP.


**Model Testing on Labeled Test Data with Performance Metrics Evaluation**

In [ ]:
import shutil

In [ ]:
# Paths
trained_model_path = "/content/runs/train/yolov8_training/weights/best.pt"  # Path to trained YOLOv8 model
test_images_path = "/content/datasets/labeled_data/test/images"  # Local path for test images
results_path = "/content/datasets/test_results_v2"  # Local folder for test results
gcs_results_path = "models/test_results/test_data_v2"  # GCS folder for saving results
test_data_yaml = "/content/test_data.yaml"  # Temporary YAML file for test data configuration


In [ ]:
# Ensure the local directory for results exists
os.makedirs(results_path, exist_ok=True)

In [ ]:
# Create a temporary data.yaml for testing
test_data_config = {
    "train": None,
    "val": test_images_path,
    "names": {
        0: "V",  # Passenger Vehicles
        1: "C",  # Cargo Vehicles
        2: "S"   # Buses
    }
}
with open(test_data_yaml, "w") as yaml_file:
    yaml.dump(test_data_config, yaml_file)

In [ ]:
# Load the trained YOLOv8 model
print("Loading the trained YOLOv8 model for testing...")
model = YOLO(trained_model_path)

Loading the trained YOLOv8 model for testing...


/usr/local/lib/python3.10/dist-packages/ultralytics/nn/tasks.py:341: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  ckpt = torch.load(attempt_download(weight), map_location='

In [ ]:
# Testing and results
try:
    # Run validation without saving the `results` object
    print("Starting testing on the labeled test data...")
    model.val(
        data=test_data_yaml,  # Use the YAML file for test data
        save_json=True,       # Save results in JSON format
        save_conf=True,       # Save confidence scores
        project=results_path, # Save results in the specified folder
        name="test_predictions",
        plots=False           # Disable plotting to avoid visualization errors
    )

    # Display validation metrics from YOLO output
    print("\nValidation completed. Check the metrics above and JSON file for detailed results.")
    print(f"JSON results saved at: {os.path.join(results_path, 'test_predictions.json')}")

except Exception as e:
    print(f"Error during validation: {e}")


Ultralytics YOLOv8.0.3 🚀 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (NVIDIA A100-SXM4-40GB, 40514MiB)
Fusing layers... 
Model summary: 168 layers, 3006233 parameters, 0 gradients, 8.1 GFLOPs


Starting testing on the labeled test data...


val: Scanning /content/datasets/labeled_data/test/labels.cache... 163 images, 0 backgrounds, 0 corrupt: 100%|██████████| 163/163 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 11/11 [00:01<00:00,  5.94it/s]
                   all        163        878      0.754      0.795      0.843      0.713
                     V        163        694      0.807      0.856      0.904      0.718
                     C        163        100      0.605       0.66      0.688      0.597
                     S        163         84      0.851      0.869      0.937      0.823
Speed: 0.1ms pre-process, 0.7ms inference, 0.0ms loss, 1.3ms post-process per image
Saving /content/datasets/test_results_v2/test_predictions/predictions.json...



Validation completed. Check the metrics above and JSON file for detailed results.
JSON results saved at: /content/datasets/test_results_v2/test_predictions.json


### Decision on Metrics

The model shows good performance overall, with an mAP@0.5 of 0.843 and mAP@0.5:0.95 of 0.713 across all classes.

- Passenger Vehicles (V): The model works well, with high precision (0.807) and recall (0.856). It also has a strong mAP@0.5 of 0.904.

- Cargo Vehicles (C): The performance for cargo vehicles is average, with precision (0.605) and recall (0.66). This suggests the need for improvement in this category.

- Buses (S): The model performs best for buses, with excellent precision (0.851) and recall (0.869). It has the highest mAP@0.5 of 0.937.

> Conclusion:
The model is suitable for detecting Passenger Vehicles and Buses, but it needs improvement for Cargo Vehicles. It can be used now, but additional data or further training might improve results for cargo vehicles.


In [ ]:
# Save testing results to GCP
print("Uploading test results to GCP...")
blobs_to_upload = [os.path.join(dp, f) for dp, dn, filenames in os.walk(results_path) for f in filenames]

for local_file in blobs_to_upload:
    relative_path = os.path.relpath(local_file, results_path)
    gcs_file_path = os.path.join(gcs_results_path, relative_path)
    blob = bucket.blob(gcs_file_path)
    blob.upload_from_filename(local_file)

print(f"Test results uploaded to GCP at: {gcs_results_path}")

Uploading test results to GCP...
Test results uploaded to GCP at: models/test_results/test_data_v2
